In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import statsmodels.formula.api as sm
import statsmodels.api as smf
from scipy.stats import gamma,norm,rayleigh,describe,scoreatpercentile

%matplotlib inline

In [2]:
from utils.Run_LCA_model import Run_LCA_model
from utils.Parameter_values_dict import Parameter_names

In [3]:
buildings_path = '..\shapefiles\\building_block_people_WGS_3_int.csv'
logistics = 'optimal'
analysis = 'Sensitivity'
n_collection = 1
n_regen = 20
for direction in ['plus', 'minus']:
    Result_Energy_m3 = pd.DataFrame()
    Result_GHG_m3 = pd.DataFrame()
    Result_COST_m3 = pd.DataFrame()
    for param in Parameter_names:
        Total_Energy_m3, Total_GHG_m3, Total_COST_m3 = Run_LCA_model(buildings_path, n_regen, n_collection, logistics, analysis, parameter = param, direction = direction)
        Total_Energy_m3['param'], Total_GHG_m3['param'], Total_COST_m3['param'] = param, param, param
        Result_Energy_m3 = Result_Energy_m3.append(Total_Energy_m3)
        Result_GHG_m3 = Result_GHG_m3.append(Total_GHG_m3)
        Result_COST_m3 = Result_COST_m3.append(Total_COST_m3)

    Result_Energy_m3["sum"] = (Result_Energy_m3.sum(axis=1)) - Result_Energy_m3["n_facilities"]
    Result_GHG_m3["sum"] = (Result_GHG_m3.sum(axis=1)) - Result_GHG_m3["n_facilities"]
    Result_COST_m3["sum"] = (Result_COST_m3.sum(axis=1)) - Result_COST_m3["n_facilities"]

    if direction == 'plus':
        Result_Energy_param_plus = Result_Energy_m3[['sum', 'param']]
        Result_GHG_param_plus = Result_GHG_m3[['sum', 'param']]
        Result_COST_param_plus = Result_COST_m3[['sum', 'param']]
    if direction == 'minus':
        Result_Energy_param_minus = Result_Energy_m3[['sum', 'param']]
        Result_GHG_param_minus = Result_GHG_m3[['sum', 'param']]
        Result_COST_param_minus = Result_COST_m3[['sum', 'param']]

In [4]:
Sensitivity_Energy = pd.merge(Result_Energy_param_plus, Result_Energy_param_minus, on = 'param')
Sensitivity_GHG = pd.merge(Result_GHG_param_plus, Result_GHG_param_minus, on = 'param')
Sensitivity_COST = pd.merge(Result_COST_param_plus, Result_COST_param_minus, on = 'param')

In [5]:
Sensitivity_Energy['Sensitivity'] = abs(Sensitivity_Energy['sum_y'] - Sensitivity_Energy['sum_x'])*5
Sensitivity_GHG['Sensitivity'] = abs(Sensitivity_GHG['sum_y'] - Sensitivity_GHG['sum_x'])*5
Sensitivity_COST['Sensitivity'] = abs(Sensitivity_COST['sum_y'] - Sensitivity_COST['sum_x'])*5

In [6]:
Sensitivity_Energy.sort(columns='Sensitivity', ascending=False)[:8]

C:\Users\Olga\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,sum_x,param,sum_y,Sensitivity
10,14.104632,adsorption_density,14.793534,3.444511
53,14.753411,sulphuric_acid_energy,14.127883,3.127643
11,14.611236,time_between_catridge_regeneration,14.270059,1.705890
9,14.305903,resin_lifetime,14.586620,1.403588
5,14.580362,resin_energy_MJ_kg,14.300932,1.397151
2,14.355511,PVC_lifetime,14.549001,0.967451
23,14.507703,transport_energy_MJ_km,14.373592,0.670555
13,14.393311,urine_production,14.503503,0.550964


In [7]:
Sensitivity_GHG.sort(columns='Sensitivity', ascending=False)[:8]

C:\Users\Olga\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,sum_x,param,sum_y,Sensitivity
13,4.036752,urine_production,4.428987,1.961176
0,2.964649,percent_served,2.688437,1.381062
54,4.284726,sulphuric_acid_GHG,4.125768,0.794789
10,4.137351,adsorption_density,4.276552,0.696003
24,4.256122,transport_GHG_kg_km,4.154372,0.508750
2,4.167599,PVC_lifetime,4.253164,0.427823
11,4.227344,time_between_catridge_regeneration,4.183151,0.220967
9,4.189005,resin_lifetime,4.222843,0.169186


In [8]:
Sensitivity_COST.sort(columns='Sensitivity', ascending=False)[:8]

C:\Users\Olga\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,sum_x,param,sum_y,Sensitivity
13,16.494009,urine_production,19.993146,17.495687
58,18.609809,sulphuric_acid_cost,17.384509,6.126501
14,17.557928,household_size,18.490354,4.662129
72,18.422707,facility_cost_regression,17.571612,4.255475
0,14.227422,percent_served,13.389233,4.190946
10,17.858000,adsorption_density,18.143306,1.426529
11,18.033420,time_between_catridge_regeneration,17.960899,0.362605
9,17.964869,resin_lifetime,18.032140,0.336357
